# Exercise-2: Decision Trees 
### Code preperations

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("../archive/data.csv")
df = df.drop(columns=['id', 'Unnamed: 32'], errors='ignore')

X = df.drop(columns=['diagnosis'])
y = df['diagnosis'].map({'M':1, 'B':0})

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

### Q2.1  Train a Decision Tree classifier using default parameters. Evaluate it on validation sets from original splits (report accuracy mean and std). 

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize Decision Tree with default parameters
clf = DecisionTreeClassifier(random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Predict on validation set
y_val_pred = clf.predict(X_val)

# Compute accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9386
